In [3]:
#Conexion a la base de datos AdventureWorks en SQL Server database
import pyodbc
import pandas as pd
import numpy as np 
connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"  # O el driver que tengas instalado
    "SERVER=ADRIANAC\SQLEXPRESS;" #Nombre o direccion IP del SQL server
    "DATABASE=AdventureWorks2022;" #Nombre de la base de datos que tenog en SQL y quiero usar
    "Trusted_Connection=yes;" #Porque tengo autenticacion de windows, sino, me pediria usuario y psw
)

# Establecer la conexión
with pyodbc.connect(connection_string) as conn:
    # Crear el cursor
    cursor = conn.cursor()

<>:7: SyntaxWarning: invalid escape sequence '\S'
<>:7: SyntaxWarning: invalid escape sequence '\S'
C:\Users\adria\AppData\Local\Temp\ipykernel_24664\3901433779.py:7: SyntaxWarning: invalid escape sequence '\S'
  "SERVER=ADRIANAC\SQLEXPRESS;" #Nombre o direccion IP del SQL server


In [4]:
#Identifica y elimina los registros duplicados de la tabla Person.Person.

#Primero realizamos la consulta para identificar los duplicados que tengamos
query = """
SELECT FirstName, LastName, MiddleName, COUNT(*) AS NumDuplicados
FROM Person.Person
GROUP BY FirstName, LastName, MiddleName
HAVING COUNT(*) > 1;
"""
# Imprimir la consulta
#print(query)
# Leer datos en un DataFrame de Pandas
df = pd.read_sql(query, conn)
# Mostrar las primeras filas
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

| FirstName   | LastName    | MiddleName   | NumDuplicados   |
|:------------|:------------|:-------------|:----------------|
| Kim         | Abercrombie |              | 2               |
| Kim         | Akers       |              | 3               |
| Michael     | Allen       |              | 3               |
| Gary        | Altman      | E.           | 2               |
| Chris       | Ashton      |              | 2               |


C:\Users\adria\AppData\Local\Temp\ipykernel_24664\1310748788.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [5]:
query="""SELECT * 
FROM Person.Person
WHERE UPPER(FirstName) LIKE UPPER('Michael') AND UPPER(LastName) LIKE UPPER('Allen'); 
"""
df = pd.read_sql(query, conn)
# Mostrar las primeras filas
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))


| BusinessEntityID   | PersonType   | NameStyle   | Title   | FirstName   | MiddleName   | LastName   | Suffix   | EmailPromotion   | AdditionalContactInfo   | Demographics                                                                                                                                                                  | rowguid                              | ModifiedDate        |
|:-------------------|:-------------|:------------|:--------|:------------|:-------------|:-----------|:---------|:-----------------|:------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------------------|:--------------------|
| 343                | SC           | False       | Mr.     | Michael     |              | Allen      |          | 2                |                         | <IndividualSurvey xmlns="http://schemas.micros

C:\Users\adria\AppData\Local\Temp\ipykernel_24664\1626439455.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [6]:
#Reemplaza los valores nulos en la columna MiddleName de la tabla Person.Person con una cadena vacía.
query=f"""
SELECT MiddleName,
ISNULL (MiddleName, '')
FROM Person.Person
"""
df = pd.read_sql(query, conn)
# Mostrar las primeras filas
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

| MiddleName   |    |
|:-------------|:---|
| E            | E  |
| R.           | R. |
| B            | B  |
|              |    |
|              |    |


C:\Users\adria\AppData\Local\Temp\ipykernel_24664\2830211442.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [7]:
#La anterior solo servia para visualizacion, aca ya hago el update en la tabla
update_query = """
UPDATE Person.Person
SET MiddleName = ''
WHERE MiddleName IS NULL;
"""

cursor.execute(update_query)
conn.commit()  # Confirmar los cambios en la base de datos

In [8]:
#Convierte la columna OrderDate de la tabla Sales.SalesOrderHeader al tipo de dato fecha.
query="""
SELECT FORMAT(OrderDate, 'dd/MM/yyyy') AS FechaNacimientoFormateada
FROM Sales.SalesOrderHeader;
"""

df = pd.read_sql(query, conn)
# Mostrar las primeras filas
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))


C:\Users\adria\AppData\Local\Temp\ipykernel_24664\2174076344.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


| FechaNacimientoFormateada   |
|:----------------------------|
| 31/05/2011                  |
| 31/05/2011                  |
| 31/05/2011                  |
| 31/05/2011                  |
| 31/05/2011                  |


In [23]:
# Consulta para recuperar datos, incluyendo la columna OrderDate y otras columnas necesarias
query = """
SELECT OrderDate, DueDate, ShipDate  -- Incluye otras columnas necesarias
FROM Sales.SalesOrderHeader;
"""
# Leer datos en un DataFrame
df = pd.read_sql(query, conn)

# Convertir las columnas de fecha a tipo DATE
df = df.assign(OrderDate = lambda x: pd.to_datetime(x['OrderDate']).dt.date)
df = df.assign(DueDate = lambda x: pd.to_datetime(x['DueDate']).dt.date)
df = df.assign(ShipDate = lambda x: pd.to_datetime(x['ShipDate']).dt.date)
# X Es el argumento que la función lambda recibirá. En este caso, x representará cada fila del DataFrame df
# x['OrderDate']: Accede al valor de la columna OrderDate en la fila actual (x).
# pd.to_datetime(): Esta función de Pandas intenta convertir el valor de OrderDate a un objeto datetime de Pandas. 
# Cambia la fecha formato
# Consulta para crear el esquema Silver
# Consulta para crear el esquema Silver
# Mostrar las primeras filas del DataFrame modificado
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))



C:\Users\adria\AppData\Local\Temp\ipykernel_30100\193837928.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


| OrderDate   | DueDate    | ShipDate   |
|:------------|:-----------|:-----------|
| 2011-05-31  | 2011-06-12 | 2011-06-07 |
| 2011-05-31  | 2011-06-12 | 2011-06-07 |
| 2011-05-31  | 2011-06-12 | 2011-06-07 |
| 2011-05-31  | 2011-06-12 | 2011-06-07 |
| 2011-05-31  | 2011-06-12 | 2011-06-07 |


In [33]:
# Consulta para crear el esquema Silver (si no existe)
create_schema_query = """
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'Silver')
BEGIN
    EXEC('CREATE SCHEMA Silver')
END
"""

# Creamos el esquema Silver
cursor.execute(create_schema_query)
conn.commit()


In [39]:

# Consulta para recuperar datos, incluyendo todas las columnas necesarias
query = """
SELECT SalesOrderID, OrderDate, DueDate, ShipDate, SalesOrderNumber, AccountNumber, CustomerID, SalesPersonID, TerritoryID, BillToAddressID, ShipToAddressID, ShipMethodID, CreditCardID, CreditCardApprovalCode, CurrencyRateID,   
 SubTotal, TaxAmt, Freight,TotalDue
FROM Sales.SalesOrderHeader;

"""

# Leer datos en un DataFrame
df = pd.read_sql(query, conn)

# Convertir las columnas de fecha a tipo DATE
df = df.assign(OrderDate=lambda x: pd.to_datetime(x['OrderDate']).dt.date)
df = df.assign(DueDate=lambda x: pd.to_datetime(x['DueDate']).dt.date)
df = df.assign(ShipDate=lambda x: pd.to_datetime(x['ShipDate']).dt.date)
# X Es el argumento que la función lambda recibirá. En este caso, x representará cada fila del DataFrame df
# x['OrderDate']: Accede al valor de la columna OrderDate en la fila actual (x).
# pd.to_datetime(): Esta función de Pandas intenta convertir el valor de OrderDate a un objeto datetime de Pandas. 
# Cambia la fecha formato
# Consulta para crear el esquema Silver
# Consulta para crear el esquema Silver
# Mostrar las primeras filas del DataFrame modificado
# Reemplazar NaN con None (para que pyodbc los interprete como NULL en SQL Server)
df = df.replace({np.nan: None})

# Mostrar las primeras filas del DataFrame modificado (opcional)
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))


# Nombre de la nueva tabla en la capa Silver
nueva_tabla_silver = 'Silver_SalesOrderHeader' 

# Obtener los nombres de las columnas del DataFrame
column_names = df.columns.tolist()

# Construir la consulta CREATE TABLE (usando las definiciones de columna proporcionadas)
create_table_query = f"""
IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[Sales].[{nueva_tabla_silver}]') AND type in (N'U'))
BEGIN
CREATE TABLE [Sales].[{nueva_tabla_silver}] ()
    SalesOrderID INT,
    OrderDate DATE,
    DueDate DATE,
    ShipDate DATE,
    SalesOrderNumber NVARCHAR(25),
    AccountNumber NVARCHAR(15),
    CustomerID INT,
    SalesPersonID INT,
    TerritoryID INT,
    BillToAddressID INT,
    ShipToAddressID INT,
    ShipMethodID INT,
    CreditCardID INT,
    CreditCardApprovalCode NVARCHAR(15),
    CurrencyRateID INT,
    SubTotal FLOAT,
    TaxAmt FLOAT,
    Freight FLOAT,
    TotalDue FLOAT
);
END
"""

# Ejecutar la consulta CREATE TABLE
cursor.execute(create_table_query)
conn.commit()

# Insertar los datos en la nueva tabla utilizando executemany (más eficiente)
values = [tuple(x) for x in df.values]
insert_query = f"INSERT INTO [Sales].[{nueva_tabla_silver}] ({', '.join(column_names)}) VALUES ({', '.join(['?'] * len(column_names))})"
cursor.executemany(insert_query, values)
conn.commit()

print(f"Datos insertados en la nueva tabla {nueva_tabla_silver}.")


C:\Users\adria\AppData\Local\Temp\ipykernel_30100\1055318402.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


| SalesOrderID   | OrderDate   | DueDate    | ShipDate   | SalesOrderNumber   | AccountNumber   | CustomerID   | SalesPersonID   | TerritoryID   | BillToAddressID   | ShipToAddressID   | ShipMethodID   | CreditCardID   | CreditCardApprovalCode   | CurrencyRateID   | SubTotal   | TaxAmt   | Freight   | TotalDue   |
|:---------------|:------------|:-----------|:-----------|:-------------------|:----------------|:-------------|:----------------|:--------------|:------------------|:------------------|:---------------|:---------------|:-------------------------|:-----------------|:-----------|:---------|:----------|:-----------|
| 43659          | 2011-05-31  | 2011-06-12 | 2011-06-07 | SO43659            | 10-4020-000676  | 29825        | 279             | 5             | 985               | 985               | 5              | 16281          | 105041Vi84182            |                  | 20565.6    | 1971.51  | 616.098   | 23153.2    |
| 43660          | 2011-05-31  | 2011-06-12 | 2011-0

KeyboardInterrupt: 

In [68]:
 #Consulta para eliminar la columna existente si existe
drop_column_query = f"""
IF EXISTS(SELECT * FROM INFORMATION_SCHEMA.COLUMNS 
          WHERE TABLE_NAME = '{nueva_tabla_silver}' AND COLUMN_NAME = 'TotalOrden')
BEGIN
    ALTER TABLE [Sales].[{nueva_tabla_silver}] 
    DROP COLUMN TotalOrden;
END
"""
cursor.execute(drop_column_query)
conn.commit()

# Nombre de la nueva tabla en la capa Silver
nueva_tabla_silver = 'Silver_SalesOrderHeader' 


# Consulta para agregar la nueva columna a la tabla
add_column_query = f"""
ALTER TABLE [Sales].[{nueva_tabla_silver}]
ADD TotalOrden FLOAT; 
"""

# Ejecutar la consulta para agregar la columna
cursor.execute(add_column_query)
conn.commit()

# Consulta para calcular el total de ventas por SalesOrderID
consulta_total_ventas = """
SELECT SalesOrderID, SUM(OrderQty * UnitPrice) AS TotalVentas
FROM Sales.SalesOrderDetail
GROUP BY SalesOrderID
"""

# Consulta UPDATE para actualizar la nueva columna con los resultados de la subconsulta
update_query = f"""
UPDATE T
SET T.TotalOrden = S.TotalVentas
FROM [Sales].[{nueva_tabla_silver}] T
INNER JOIN ({consulta_total_ventas}) S 
ON T.SalesOrderID = S.SalesOrderID;
"""

try:
    cursor.execute(update_query)
    conn.commit()
    print(f"Columna 'TotalOrden' agregada y actualizada en la tabla '{nueva_tabla_silver}'.")
except pyodbc.Error as err:
    print("Error al actualizar la tabla:", err)
    conn.rollback() 

Columna 'TotalOrden' agregada y actualizada en la tabla 'Silver_SalesOrderHeader'.


In [61]:
#Crea una nueva columna en la tabla Sales.SilverSalesOrderHeader que calcule el total de la orden multiplicando OrderQty y UnitPrice para cada fila.

query_CTE = f""" 
WITH TotalOrder AS(
SELECT ssod.SalesOrderID, SUM(sod.OrderQty*sod.UnitPrice) AS TotalSumOrder
FROM Sales.Silver_SalesOrderHeader ssod
LEFT JOIN Sales.SalesOrderDetail sod ON ssod.SalesOrderID = sod.SalesOrderID
GROUP BY ssod.SalesOrderID 
)

SELECT ssod.*, tor.TotalSumOrder
FROM Sales.Silver_SalesOrderHeader ssod
LEFT JOIN TotalOrder tor ON ssod.SalesOrderID = tor.SalesOrderID
"""

# Leer datos en un DataFrame
df = pd.read_sql(query_CTE, conn)
# Mostrar las primeras filas del DataFrame modificado (opcional)
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))




C:\Users\adria\AppData\Local\Temp\ipykernel_30100\2039030790.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_CTE, conn)


| SalesOrderID   | OrderDate   | DueDate    | ShipDate   | SalesOrderNumber   | AccountNumber   | CustomerID   | SalesPersonID   | TerritoryID   | BillToAddressID   | ShipToAddressID   | ShipMethodID   | CreditCardID   | CreditCardApprovalCode   | CurrencyRateID   | SubTotal   | TaxAmt   | Freight   | TotalDue   | TotalSumOrder   |
|:---------------|:------------|:-----------|:-----------|:-------------------|:----------------|:-------------|:----------------|:--------------|:------------------|:------------------|:---------------|:---------------|:-------------------------|:-----------------|:-----------|:---------|:----------|:-----------|:----------------|
| 43659          | 2011-05-31  | 2011-06-12 | 2011-06-07 | SO43659            | 10-4020-000676  | 29825        | 279             | 5             | 985               | 985               | 5              | 16281          | 105041Vi84182            | nan              | 20565.6    | 1971.51  | 616.098   | 23153.2    | 20565.6         

In [7]:
#Utiliza una función lambda para extraer el año de la columna OrderDate y crear una nueva columna llamada OrderYear.
# Consulta para recuperar datos

query = """
SELECT OrderDate  -- Incluye otras columnas necesarias
FROM Sales.Silver_SalesOrderHeader;
"""
# Leer datos en un DataFrame
df = pd.read_sql(query, conn)
# Convertir explícitamente la columna OrderDate a datetime
df['OrderDate'] = pd.to_datetime(df['OrderDate'])
# Extraer el año de OrderDate y crear la columna OrderYear
df = df.assign(OrderYear=lambda x: x['OrderDate'].dt.year)

# Mostrar las primeras filas del DataFrame modificado (opcional)
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

| OrderDate           | OrderYear   |
|:--------------------|:------------|
| 2011-05-31 00:00:00 | 2011        |
| 2011-05-31 00:00:00 | 2011        |
| 2011-05-31 00:00:00 | 2011        |
| 2011-05-31 00:00:00 | 2011        |
| 2011-05-31 00:00:00 | 2011        |


C:\Users\adria\AppData\Local\Temp\ipykernel_22796\2972728285.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [9]:
#Filtra las órdenes de venta que se realizaron en el año 2014.

query = f"""
SELECT SalesOrderID AS OrdenVenta
FROM Sales.Silver_SalesOrderHeader
WHERE OrderDate LIKE '2014%'
"""
# Leer datos en un DataFrame
df = pd.read_sql(query, conn)


# Mostrar las primeras filas del DataFrame modificado (opcional)
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

| OrdenVenta   |
|:-------------|
| 63363        |
| 63364        |
| 63365        |
| 63366        |
| 63367        |


C:\Users\adria\AppData\Local\Temp\ipykernel_22796\3196187427.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [7]:
#Une las tablas Sales.SalesOrderHeader y Sales.SalesOrderDetail utilizando la columna SalesOrderID.
query = """
SELECT *  
FROM Sales.SalesOrderHeader;
"""
query1 = """
SELECT *  
FROM Sales.SalesOrderDetail;
"""

# Leer los datos en un DataFrame de Pandas
df_sales_order_header = pd.read_sql(query, conn) 
df_sales_order_detail = pd.read_sql(query1,conn)
# Unión con FULL JOIN
df_merged_full = pd.merge(df_sales_order_header, df_sales_order_detail, on='SalesOrderID', how='outer')

# Mostrar las primeras filas del DataFrame modificado (opcional)
print(df_merged_full.head().to_markdown(index=False, numalign="left", stralign="left"))

C:\Users\adria\AppData\Local\Temp\ipykernel_5312\230464771.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sales_order_header = pd.read_sql(query, conn)
C:\Users\adria\AppData\Local\Temp\ipykernel_5312\230464771.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sales_order_detail = pd.read_sql(query1,conn)


| SalesOrderID   | RevisionNumber   | OrderDate           | DueDate             | ShipDate            | Status   | OnlineOrderFlag   | SalesOrderNumber   | PurchaseOrderNumber   | AccountNumber   | CustomerID   | SalesPersonID   | TerritoryID   | BillToAddressID   | ShipToAddressID   | ShipMethodID   | CreditCardID   | CreditCardApprovalCode   | CurrencyRateID   | SubTotal   | TaxAmt   | Freight   | TotalDue   | Comment   | rowguid_x                            | ModifiedDate_x      | SalesOrderDetailID   | CarrierTrackingNumber   | OrderQty   | ProductID   | SpecialOfferID   | UnitPrice   | UnitPriceDiscount   | LineTotal   | rowguid_y                            | ModifiedDate_y      |
|:---------------|:-----------------|:--------------------|:--------------------|:--------------------|:---------|:------------------|:-------------------|:----------------------|:----------------|:-------------|:----------------|:--------------|:------------------|:------------------|:---------------|:-

In [12]:
#Realiza un LEFT JOIN entre las tablas Person.Person y HumanResources.Employee para obtener información adicional de los empleados.
# Consultas SQL para obtener los datos (seleccionando columnas específicas)
query2 = """
SELECT BusinessEntityID, FirstName, LastName 
FROM Person.Person;
"""
query3 = """
SELECT BusinessEntityID, JobTitle, HireDate
FROM HumanResources.Employee;
"""

# Leer los datos en un DataFrame de Pandas
df_Person = pd.read_sql(query2, conn) 
df_Employee = pd.read_sql(query3,conn)
# Unión con LEFT JOIN
df_merged_left = pd.merge(df_Person, df_Employee, on='BusinessEntityID', how='left')

# Mostrar las primeras filas del DataFrame modificado (opcional)
print(df_merged_left.head().to_markdown(index=False, numalign="left", stralign="left"))

| BusinessEntityID   | FirstName   | LastName    | JobTitle                     | HireDate   |
|:-------------------|:------------|:------------|:-----------------------------|:-----------|
| 285                | Syed        | Abbas       | Pacific Sales Manager        | 2013-03-14 |
| 293                | Catherine   | Abel        | nan                          | nan        |
| 38                 | Kim         | Abercrombie | Production Technician - WC60 | 2010-01-16 |
| 295                | Kim         | Abercrombie | nan                          | nan        |
| 2170               | Kim         | Abercrombie | nan                          | nan        |


C:\Users\adria\AppData\Local\Temp\ipykernel_5312\1989406769.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_Person = pd.read_sql(query2, conn)
C:\Users\adria\AppData\Local\Temp\ipykernel_5312\1989406769.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_Employee = pd.read_sql(query3,conn)


In [16]:
#Calcula el total de ventas por empleado uniendo las tablas Sales.SalesOrderHeader, Sales.SalesOrderDetail y HumanResources.Employee.
# Consultas SQL para obtener los datos de las tres tablas
query_sales_order_header = """
SELECT SalesOrderID, SalesPersonID
FROM Sales.SalesOrderHeader;
"""

query_sales_order_detail = """
SELECT SalesOrderID, OrderQty, UnitPrice
FROM Sales.SalesOrderDetail;
"""

query_employee = """
SELECT e.BusinessEntityID, FirstName, LastName
FROM HumanResources.Employee e
INNER JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID;
"""

# Leer los datos en DataFrames de Pandas
df_sales_order_header = pd.read_sql(query_sales_order_header, conn)
df_sales_order_detail = pd.read_sql(query_sales_order_detail, conn)
df_employee = pd.read_sql(query_employee, conn)

# Unir las tablas SalesOrderHeader y SalesOrderDetail
df_joined = pd.merge(df_sales_order_header, df_sales_order_detail, on='SalesOrderID', how='inner')

# Unir el resultado anterior con la tabla Employee
df_final = pd.merge(df_joined, df_employee, left_on='SalesPersonID', right_on='BusinessEntityID', how='inner')

# Calcular el total de ventas por empleado
df_total_ventas = df_final.groupby(['BusinessEntityID', 'FirstName', 'LastName']).agg(
    TotalVentas=('OrderQty', lambda x: (x * df_final['UnitPrice']).sum())
).reset_index()

# Mostrar el resultado sin notación científica
print(df_total_ventas.head().to_markdown(index=False, numalign="left", stralign="left", floatfmt='.2f'))



C:\Users\adria\AppData\Local\Temp\ipykernel_5312\3775163763.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sales_order_header = pd.read_sql(query_sales_order_header, conn)
C:\Users\adria\AppData\Local\Temp\ipykernel_5312\3775163763.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sales_order_detail = pd.read_sql(query_sales_order_detail, conn)


| BusinessEntityID   | FirstName   | LastName   | TotalVentas   |
|:-------------------|:------------|:-----------|:--------------|
| 274                | Stephen     | Jiang      | 1111762.00    |
| 275                | Michael     | Blythe     | 9324137.16    |
| 276                | Linda       | Mitchell   | 10444625.80   |
| 277                | Jillian     | Carson     | 10095040.78   |
| 278                | Garrett     | Vargas     | 3631749.85    |


C:\Users\adria\AppData\Local\Temp\ipykernel_5312\3775163763.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_employee = pd.read_sql(query_employee, conn)
